In [30]:
#Download ctrNet-tool 
#You can find the code in https://github.com/guoday/ctrNet-tool
!git clone https://github.com/guoday/ctrNet-tool.git
!cp -r ctrNet-tool/* ./
# !rm -r ctrNet-tool data .git
!ls -all

fatal: destination path 'ctrNet-tool' already exists and is not an empty directory.
total 415920
drwxr-xr-x 14 d d      4096 Feb 21 07:01 .
drwxr-xr-x  5 d d      4096 Feb 20 00:22 ..
-rw-rw-r--  1 d d   4440024 Feb 20 00:25 3_blend.csv
-rw-rw-r--  1 d d     10603 Feb 20 00:25 belnd.ipynb
-rw-rw-r--  1 d d    469148 Feb 20 00:14 best_cv.ipynb
-rw-r--r--  1 d d    658463 Feb 13 22:56 best_single_model2.ipynb
-rw-r--r--  1 d d    657844 Feb 17 06:29 best_single_model-Copy3.ipynb
-rw-r--r--  1 d d    657844 Feb 18 23:30 best_single_model.ipynb
-rw-rw-r--  1 d d     61461 Feb 17 02:48 best_single_model-tfidf.ipynb
-rw-rw-r--  1 d d    639615 Feb 12 21:21 best_single_model-分类-Copy1.ipynb
-rw-r--r--  1 d d    668175 Feb 14 01:27 best_single_model-分类.ipynb
-rw-r--r--  1 d d    685859 Feb 12 08:23 best_single_model-去除异常点.ipynb
-rw-rw-r--  1 d d 409991896 Feb 20 05:02 best_t_weight.h5
drwxr-xr-x  3 d d      4096 Feb 20 05:08 class_oof
drwxr-xr-x  3 d d      4096 Feb 19 10:41 class_sub
-rw-rw-r-

In [31]:
import ctrNet
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from src import misc_utils as utils
import os
import gc


DEBUG = False

NROWS = None

if DEBUG:
    NROWS = 1000

In [32]:
import numpy as np
import pandas as pd
import datetime
from datetime import date, datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings

# import workalendar
# from workalendar.america import Brazil

warnings.filterwarnings('ignore')
np.random.seed(4590)

skf= StratifiedKFold(n_splits=5, shuffle=True, random_state=4590)

# by feature select 
# FILTER_FEATURE = ['new_hist_month_diff_min_-_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean_add_hist_merchant_category_id_mean_mean', 'new_hist_month_lag_mean_mean_add_hist_month_lag_mean_mean', 'hist_year_nunique', 'new_hist_month_diff_max_/_hist_month_diff_max', 'new_hist_merchant_id_mean_mean_add_hist_merchant_id_mean_mean', 'new_hist_month_diff_min_/_hist_month_diff_min', 'new_hist_year_mean_mean_add_hist_year_mean_mean', 'new_hist_dayofweek_mean_mean_/_hist_dayofweek_mean_mean', 'new_hist_weekofyear_mean_mean_add_hist_weekofyear_mean_mean', 'new_hist_dayofweek_nunique_/_hist_dayofweek_nunique', 'new_hist_month_diff_max', 'new_hist_year_mean_mean_/_hist_year_mean_mean', 'new_hist_merchant_category_id_mean_mean_/_hist_merchant_category_id_mean_mean', 'new_hist_merchant_id_mean_mean_/_hist_merchant_id_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean', 'hist_month_diff_mean_hist_month_diff_min_ctr', 'new_hist_year_nunique_-_hist_year_nunique', 'dayofweek', 'new_hist_month_mean_mean_/_hist_month_mean_mean', 'new_hist_month_diff_var', 'new_hist_installments_min_-_hist_installments_min', 'new_hist_subsector_id_mean_mean', 'new_hist_dayofweek_nunique', 'new_hist_weekend_mean', 'new_hist_category_2_mean_mean_/_hist_category_2_mean_mean', 'new_hist_merchant_category_id_nunique_-_hist_merchant_category_id_nunique', 'new_hist_month_diff_mean_add_hist_month_diff_mean', 'new_hist_authorized_flag_mean', 'new_hist_merchant_category_id_mean_mean_-_hist_merchant_category_id_mean_mean', 'new_hist_year_mean_mean_-_hist_year_mean_mean', 'new_hist_category_2_mean_mean', 'new_hist_card_id_mean_mean_add_hist_card_id_mean_mean', 'new_hist_month_diff_max_-_hist_month_diff_max', 'new_hist_weekend_sum']

In [33]:
df_train = pd.read_csv('../input/train.csv',nrows = NROWS)
df_test = pd.read_csv('../input/test.csv',nrows = NROWS)
df_hist_trans = pd.read_csv('../input/historical_transactions.csv',parse_dates=['purchase_date'], nrows = NROWS)
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv',parse_dates=['purchase_date'], nrows = NROWS)

In [34]:
max_date=df_new_merchant_trans['purchase_date'].max()

In [35]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [36]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)
gc.collect()

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


83400

In [37]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988
3,2017-09,C_ID_186d6a6901,4,3,0,0.142456
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159790


In [38]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [39]:

for df in [df_hist_trans,df_new_merchant_trans]:
#     df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    df['month_diff'] = ((max_date - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
#     df['authorized_flag_purchase_amount'] = df.apply(lambda x: x['purchase_amount'] \
#                             if x['authorized_flag']>0 else 0 ,axis=1)
    
#     df['unauthorized_flag_purchase_amount'] = df.apply(lambda x: x['purchase_amount'] \
#                             if x['authorized_flag']<1 else 0 ,axis=1)
    
    
    
    
    
                                                                           

In [40]:
# df_hist_trans.groupby('card_id')['month'].unique()

In [41]:
def  get_agg_fea(count_df,prefix):
    aggs = {}
    for col in ['month','day','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var']
    aggs['installments'] = ['sum','max','min','mean','var']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var']
    # aggs['month_diff'] = ['mean']
    aggs['month_diff'] = ['max','min','mean','var']
    aggs['authorized_flag'] = ['sum', 'mean']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']

    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',\
                'category_1','category_2','category_3','month_lag','card_id']:
        count_df[col+'_mean'] = count_df.groupby([col])['purchase_amount'].transform('mean')
        aggs[col+'_mean'] = ['mean']   

    new_columns = get_new_columns(prefix,aggs)
    count_df_gp = count_df.groupby('card_id').agg(aggs)
    count_df_gp.columns = new_columns
    count_df_gp.reset_index(drop=False,inplace=True)
    count_df_gp['%s_purchase_date_diff'%prefix] = (count_df_gp['%s_purchase_date_max'%prefix] - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    count_df_gp['%s_purchase_date_average'%prefix] = count_df_gp['%s_purchase_date_diff'%prefix]/count_df_gp['%s_card_id_size'%prefix]
    count_df_gp['%s_purchase_date_uptonow'%prefix] = (max_date - count_df_gp['%s_purchase_date_max'%prefix]).dt.days
    count_df_gp['%s_purchase_date_uptomin'%prefix] = (max_date - count_df_gp['%s_purchase_date_min'%prefix]).dt.days
    
    return count_df_gp



hist_count_df=get_agg_fea(df_hist_trans,'hist')
df_train = df_train.merge(hist_count_df,on='card_id',how='left')
df_test = df_test.merge(hist_count_df,on='card_id',how='left')
del hist_count_df
gc.collect()
new_count_df = get_agg_fea(df_new_merchant_trans,'new_hist')
df_train = df_train.merge(new_count_df,on='card_id',how='left')
df_test = df_test.merge(new_count_df,on='card_id',how='left')
del new_count_df
gc.collect()

157

In [42]:
for df in [df_train,df_test]:
    
    
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['first_active_monthtonoew'] = ( max_date-pd.to_datetime(df['first_active_month'])).dt.days

    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (max_date- df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['hist_sleep_day'] = df['first_active_monthtonoew'] - df['hist_day_nunique']
    df['new_sleep_day'] = df['first_active_monthtonoew'] - df['new_hist_day_nunique']
    df['total_sleep_day'] = df['hist_sleep_day'] + df['new_sleep_day'] 
    df['diff_sleep_day'] = df['hist_sleep_day'] - df['new_sleep_day']
   
    
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']

    
    df['purchase_amount_diff'] = df['new_hist_purchase_amount_sum']-df['hist_purchase_amount_sum']
    df['purchase_amount_rate'] = df['purchase_amount_diff']/df['hist_purchase_amount_sum']

  

In [43]:
new_hist_feature=[ i for i in df_train  if 'new_hist' in i]

for fea in new_hist_feature:
    if fea in ['new_hist_purchase_date_max','new_hist_purchase_date_min']:
        continue
    print(fea)
    new_fea=fea
    hist_fea=fea.replace("new_hist",'hist')
    df_train[new_fea+"_add_"+hist_fea]=df_train[new_fea]+df_train[hist_fea]
    df_test[new_fea+"_add_"+hist_fea]=df_test[new_fea]+df_test[hist_fea]
    df_train[new_fea+"_-_"+hist_fea]=df_train[new_fea]-df_train[hist_fea]
    df_test[new_fea+"_-_"+hist_fea]=df_test[new_fea]-df_test[hist_fea]
    df_train[new_fea+"_/_"+hist_fea]=df_train[new_fea]/(df_train[hist_fea]+0.01)
    df_test[new_fea+"_/_"+hist_fea]=df_test[new_fea]/(df_test[hist_fea]+0.01)

new_hist_month_nunique
new_hist_day_nunique
new_hist_hour_nunique
new_hist_weekofyear_nunique
new_hist_dayofweek_nunique
new_hist_year_nunique
new_hist_subsector_id_nunique
new_hist_merchant_id_nunique
new_hist_merchant_category_id_nunique
new_hist_purchase_amount_sum
new_hist_purchase_amount_max
new_hist_purchase_amount_min
new_hist_purchase_amount_mean
new_hist_purchase_amount_var
new_hist_installments_sum
new_hist_installments_max
new_hist_installments_min
new_hist_installments_mean
new_hist_installments_var
new_hist_month_lag_max
new_hist_month_lag_min
new_hist_month_lag_mean
new_hist_month_lag_var
new_hist_month_diff_max
new_hist_month_diff_min
new_hist_month_diff_mean
new_hist_month_diff_var
new_hist_authorized_flag_sum
new_hist_authorized_flag_mean
new_hist_weekend_sum
new_hist_weekend_mean
new_hist_category_1_sum
new_hist_category_1_mean
new_hist_card_id_size
new_hist_month_mean_mean
new_hist_hour_mean_mean
new_hist_weekofyear_mean_mean
new_hist_dayofweek_mean_mean
new_hist_yea

In [44]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

0    199710
1      2207
Name: outliers, dtype: int64

In [45]:
df_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target,hist_month_nunique,hist_day_nunique,hist_hour_nunique,hist_weekofyear_nunique,...,new_hist_purchase_date_uptonow_add_hist_purchase_date_uptonow,new_hist_purchase_date_uptonow_-_hist_purchase_date_uptonow,new_hist_purchase_date_uptonow_/_hist_purchase_date_uptonow,new_hist_purchase_date_uptomin_add_hist_purchase_date_uptomin,new_hist_purchase_date_uptomin_-_hist_purchase_date_uptomin,new_hist_purchase_date_uptomin_/_hist_purchase_date_uptomin,new_hist_first_buy_add_hist_first_buy,new_hist_first_buy_-_hist_first_buy,new_hist_first_buy_/_hist_first_buy,outliers
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820312,9,31,23,35,...,65.0,-63.0,0.015623,363.0,-251.0,0.182404,303.0,251.0,10.649750,0
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392822,12,31,24,50,...,120.0,-58.0,0.348275,567.0,-391.0,0.183712,401.0,391.0,79.041916,0
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.687988,10,19,14,22,...,64.0,-60.0,0.032253,476.0,-472.0,0.004219,798.0,472.0,3.895467,0
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142456,6,25,16,20,...,73.0,-49.0,0.196689,270.0,-162.0,0.249988,212.0,162.0,7.477009,0
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159790,4,30,22,17,...,63.0,-59.0,0.032782,228.0,-110.0,0.349092,132.0,110.0,10.990009,0


In [46]:
# Dealing with the one nan in df_test.first_active_month a bit arbitrarily for now
try:
    df_test.loc[df_test['first_active_month'].isna(),'first_active_month'] = df_test.iloc[11577]['first_active_month']
except:
    pass

In [47]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 123.63 Mb (64.4% reduction)
Mem. usage decreased to 73.57 Mb (65.2% reduction)


In [48]:
bed_fea = ['new_hist_month_diff_min_-_hist_month_diff_min', 'new_hist_year_mean_mean_/_hist_year_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_year_mean_mean_add_hist_year_mean_mean', 'new_hist_merchant_category_id_mean_mean_-_hist_merchant_category_id_mean_mean', 'new_hist_month_lag_mean_mean_-_hist_month_lag_mean_mean', 'new_hist_month_diff_max_add_hist_month_diff_max', 'new_hist_month_diff_min', 'new_hist_weekend_sum_/_hist_weekend_sum', 'new_hist_year_mean_mean', 'new_hist_merchant_id_mean_mean_add_hist_merchant_id_mean_mean', 'new_hist_purchase_amount_sum_-_hist_purchase_amount_sum', 'new_hist_month_diff_var', 'new_hist_month_diff_max_/_hist_month_diff_max', 'new_hist_weekofyear_nunique', 'new_hist_month_diff_max_-_hist_month_diff_max', 'new_hist_weekend_mean_add_hist_weekend_mean', 'new_hist_month_mean_mean_-_hist_month_mean_mean', 'new_hist_month_diff_mean', 'hist_year_nunique', 'new_hist_weekend_mean', 'new_hist_year_mean_mean_-_hist_year_mean_mean', 'hist_month_diff_mean_hist_month_diff_min_ctr', 'hist_dayofweek_nunique', 'new_hist_merchant_category_id_mean_mean', 'new_hist_weekend_sum', 'new_hist_installments_min_-_hist_installments_min', 'new_hist_authorized_flag_mean', 'new_hist_weekend_mean_/_hist_weekend_mean', 'hist_month_diff_min', 'new_hist_card_id_mean_mean_add_hist_card_id_mean_mean', 'new_hist_month_mean_mean_/_hist_month_mean_mean', 'new_hist_category_2_mean_mean', 'new_hist_month_diff_min_/_hist_month_diff_min', 'new_hist_merchant_category_id_mean_mean_add_hist_merchant_category_id_mean_mean', 'new_hist_dayofweek_nunique_/_hist_dayofweek_nunique', 'new_hist_month_mean_mean', 'new_hist_day_nunique_add_hist_day_nunique', 'new_hist_subsector_id_nunique_-_hist_subsector_id_nunique', 'new_hist_purchase_amount_max_add_hist_purchase_amount_max', 'new_hist_month_diff_mean_add_hist_month_diff_mean', 'new_hist_subsector_id_mean_mean', 'new_hist_year_nunique_-_hist_year_nunique', 'new_hist_dayofweek_nunique_-_hist_dayofweek_nunique', 'new_hist_merchant_id_mean_mean_/_hist_merchant_id_mean_mean']

In [49]:
# bed_fea =['new_hist_month_diff_min', 'new_hist_authorized_flag_mean', 'new_hist_year_mean_mean', 'new_hist_year_nunique_/_hist_year_nunique', 'new_hist_year_nunique_-_hist_year_nunique']
del_col =  ['card_id', 'first_active_month','target','outliers']+bed_fea

features = [c for c in df_train.columns if c not in del_col ]

target = df_train['outliers']

train_y = target 
# df_train.drop(columns  = ["card_id","target"],inplace=True)
# df_test.drop(columns = ["card_id"],inplace=True)

In [50]:
print(len(features))

235


In [51]:
features

['feature_1',
 'feature_2',
 'feature_3',
 'hist_month_nunique',
 'hist_day_nunique',
 'hist_hour_nunique',
 'hist_weekofyear_nunique',
 'hist_subsector_id_nunique',
 'hist_merchant_id_nunique',
 'hist_merchant_category_id_nunique',
 'hist_purchase_amount_sum',
 'hist_purchase_amount_max',
 'hist_purchase_amount_min',
 'hist_purchase_amount_mean',
 'hist_purchase_amount_var',
 'hist_installments_sum',
 'hist_installments_max',
 'hist_installments_min',
 'hist_installments_mean',
 'hist_installments_var',
 'hist_purchase_date_max',
 'hist_purchase_date_min',
 'hist_month_lag_max',
 'hist_month_lag_min',
 'hist_month_lag_mean',
 'hist_month_lag_var',
 'hist_month_diff_max',
 'hist_month_diff_mean',
 'hist_month_diff_var',
 'hist_authorized_flag_sum',
 'hist_authorized_flag_mean',
 'hist_weekend_sum',
 'hist_weekend_mean',
 'hist_category_1_sum',
 'hist_category_1_mean',
 'hist_card_id_size',
 'hist_month_mean_mean',
 'hist_hour_mean_mean',
 'hist_weekofyear_mean_mean',
 'hist_dayofweek_m

In [52]:
hparam=tf.contrib.training.HParams(
            model='nffm',
            norm=True,
            batch_norm_decay=0.9,
            hidden_size=[128,128],
            k=16,
            hash_ids=int(2e4),
            batch_size= 129,
            optimizer="adam",
            learning_rate=0.001,
            num_display_steps=1000,
            num_eval_steps=1000,
            epoch=1,
            metric= 'logloss',
            init_method='uniform',
            init_value=0.1,
            feature_nums=len(features))
utils.print_hparams(hparam)

  batch_norm_decay=0.9
  batch_size=129
  epoch=1
  feature_nums=235
  hash_ids=20000
  hidden_size=[128, 128]
  init_method=uniform
  init_value=0.1
  k=16
  learning_rate=0.001
  metric=logloss
  model=nffm
  norm=True
  num_display_steps=1000
  num_eval_steps=1000
  optimizer=adam


In [53]:
len(features)

235

In [54]:
# float_features = []

# for i in  features:
#     df_train[i].fillna(1e10,inplace=True)
#     df_test[i].fillna(1e10,inplace=True)
#     df_train[i] = df_train[i].apply(lambda x: round(x)).astype(int)
#     df_test[i] = df_test[i].apply(lambda x: round(x)).astype(int)
#     print(i,df_train[i].dtype)

In [55]:
##分桶
def make_bucket(data,num=10):
    data.sort()
    bins=[]
    for i in range(num):
        bins.append(data[int(len(data)*(i+1)//num)-1])
    return bins


for f in features:
    df_train[f]=df_train[f].fillna(1e10)
    df_test[f]=df_test[f].fillna(1e10)
    data=list(df_train[f])+list(df_test[f])
    bins=make_bucket(data,num=1000)
    df_train[f]=np.digitize(df_train[f],bins=bins)
    df_test[f]=np.digitize(df_test[f],bins=bins)
    


In [56]:
df_test['outliers'] = 0  #郭大代码的bug 必须传个label

In [57]:
from sklearn.model_selection import  StratifiedKFold
from sklearn import  metrics

skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=4590)
oof_train = np.zeros((len(df_train),1))
oof_test = np.zeros((len(df_test),1))
score_csv = []
for fold,(trn_idx,val_idx) in enumerate (skf.split(df_train,df_train['outliers'])):
    print("开始第 %d 折训练"%(fold+1))
    x_tr,x_val = df_train.iloc[trn_idx][features],df_train.iloc[val_idx][features]
    y_tr,y_val = df_train.iloc[trn_idx]['outliers'],df_train.iloc[val_idx]['outliers']
    model = ctrNet.build_model(hparam)
    model.train(train_data = (x_tr,y_tr),dev_data=(x_val,y_val))
    del x_tr,y_tr
    gc.collect()
    predictions = model.infer(dev_data=(x_val,y_val))
    oof_train[val_idx] = predictions.reshape(-1,1)
    false_positive_rate, recall, thresholds = metrics.roc_curve(y_val, predictions)
    score = metrics.auc(false_positive_rate, recall)
    print("auc score %s"%score)
    score_csv.append(score)
    del x_val,y_val
    gc.collect()
    oof_test = model.infer(dev_data=(df_test[features],df_test['outliers'])).reshape(-1,1)/5
    
mean_score = np.mean(score_csv)
print("mean score %s"%mean_score)

开始第 1 折训练
# Trainable variables
  emb_v1:0, (20000, 1), 
  emb_v2:0, (20000, 235, 16), 
  Variable:0, (27495, 128), 
  norm_0/beta:0, (128,), 
  norm_0/gamma:0, (128,), 
  Variable_1:0, (128, 128), 
  norm_1/beta:0, (128,), 
  norm_1/gamma:0, (128,), 
  Variable_2:0, (128, 1), 
  Variable_3:0, (), 
  epoch 0 step 1000 lr 0.001 logloss 0.049089 gN 0.13, Thu Feb 21 07:14:50 2019
# Epcho-time 260.28s Eval logloss 0.045482. Best logloss 0.045482.
# Epcho-time 367.80s Eval logloss 0.082793. Best logloss 0.045482.
INFO:tensorflow:Restoring parameters from model_tmp/model
# Epcho-time 410.14s Eval logloss 0.078544. Best logloss 0.045482.
auc score 0.45777992908722176
开始第 2 折训练
# Trainable variables
  emb_v1:0, (20000, 1), 
  emb_v2:0, (20000, 235, 16), 
  Variable:0, (27495, 128), 
  norm_0/beta:0, (128,), 
  norm_0/gamma:0, (128,), 
  Variable_1:0, (128, 128), 
  norm_1/beta:0, (128,), 
  norm_1/gamma:0, (128,), 
  Variable_2:0, (128, 1), 
  Variable_3:0, (), 
  epoch 0 step 1000 lr 0.001 lo

In [ ]:
train_prob=pd.DataFrame(oof_train)
train_prob.columns=['class1']
train_prob.to_csv("class_oof/train_prob_%s.csv"%mean_loss,index=False)

test_prob=pd.DataFrame(oof_test)
test_prob.columns=['class1']
test_prob.to_csv("class_oof/test_prob_%s.csv"%mean_loss,index=False)

np.save("train_y",target)